In [1]:
# alpha: initial import statements
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import minmax_scale as mms
# !pip install category_encoders
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.svm.libsvm import predict
import random

In [2]:
# bravo: give background, then load and inspect data
# here's a URL for background: https://www.kaggle.com/t/32b89e93d8a44743983a0ab1c19c85f3
# Q: "Can you predict which [Tanzanian] water pumps are faulty?"
# H_null: something like "all features are equally likely to make H2O unclean and/or non-potable"
# H_one: akin to "one or a group of features enables high-accuracy prediction of faulty pumps"

kaggle_path = 'C:\\Users\\jhump\\Desktop\\Desktop_professional\\LSDS\\Full_Course\\train_features.csv'

df = pd.read_csv(kaggle_path)
print(df.shape)
df.head()

(59400, 40)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [4]:
# charlie: check data for nan's
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [5]:
df1 = df.copy()

# source for following: https://stackoverflow.com/questions/22470690/get-list-of-pandas-dataframe-columns-based-on-data-type
df1_dtype_groups = df1.columns.to_series().groupby(df1.dtypes).groups
df1_dtype_groups

{dtype('int64'): Index(['id', 'gps_height', 'num_private', 'region_code', 'district_code',
        'population', 'construction_year'],
       dtype='object'),
 dtype('float64'): Index(['amount_tsh', 'longitude', 'latitude'], dtype='object'),
 dtype('O'): Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
        'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
        'scheme_management', 'scheme_name', 'permit', 'extraction_type',
        'extraction_type_group', 'extraction_type_class', 'management',
        'management_group', 'payment', 'payment_type', 'water_quality',
        'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
        'source_class', 'waterpoint_type', 'waterpoint_type_group'],
       dtype='object')}

In [6]:
# what features will show whether or not H2O is clean and potable?
# let's investigate 'water_quality'
df1.water_quality.unique()  # questions: merge both 'abandoned' strings into one? what to do with 'unknown'?

array(['soft', 'salty', 'milky', 'unknown', 'fluoride', 'coloured',
       'salty abandoned', 'fluoride abandoned'], dtype=object)

In [7]:
# delta: preprocess data
# CREDIT for this function, to Ryan Herr/LSDS

def train_validation_test_split(X, y, train_size=0.8, val_size=0.1, test_size=0.1, 
                                random_state=None, shuffle=True):
    assert train_size + val_size + test_size == 1
    
    X_train_val, X_test, y_train_val, y_test = tts(X, y, test_size=test_size,
                                                   random_state=random_state, shuffle=shuffle)
    
    X_train, X_val, y_train, y_val = tts(X_train_val, y_train_val,
                                         test_size=val_size/(train_size+val_size), 
                                         random_state=random_state, shuffle=shuffle)
    print('X_train is:', X_train, '\n')
    return X_train, X_val, X_test, y_train, y_val, y_test

In [8]:
# echo: fit, validate, and present model

# 2019-02-04 comment: I believe that some sort of advanced regression will be best here, but TBD
# for baseline model, X might include 'id', 'region_code', 'population', 'construction_year'
# IMPORTANT NOTE: 2019-02-06 0830hrs PST: REFACTOR y

train_labels = pd.read_csv('C:\\Users\\jhump\\Desktop\\Desktop_professional\\LSDS\\train_labels.csv')
df2 = train_labels.copy()

# X = df1[['id', 'region_code', 'population', 'construction_year']]
# got much, much lower accuracy score, after refactoring, so will start to 'twiddle' the feature knobs
# first, take out 'id' -> accuracy score: slightly increased, at _0.5579545454545455_
# X = df1[['region_code', 'population', 'construction_year']]

# second, add in 'gps_height' -> accuracy score: not significantly changed
# X = df1[['gps_height', 'region_code', 'population', 'construction_year']]

# third, take 'gps_height' back out, and take out 'region_code' -> accuracy score: 0.5430134680134681
# X = df1[['population', 'construction_year']]
# KEY: thus, need to add in categorical features--onehotencoding as needed

In [9]:
# continue fitting, validating LogReg model
# now, after those 3 trials above, will add 'water_quality' to X -> accuracy score:
X = df1[['population', 'construction_year', 'water_quality']]

# call 'mms' on X after creating X namespace
# mms(X) -- 2019-02-06 0935hrs: attempting pipeline for first time
y = df2.status_group

# X_train, X_test, y_train, y_test = tts(X, y, train_size=0.6, random_state=42)

# log_reg = LogisticRegression().fit(X, y)
# log_reg.fit(X_train, y_train)
# y_pred = log_reg.predict(X_test)
# accuracy_score(y_test, y_pred)

# attempt this after 60% accuracy achieved
train_validation_test_split(X, y)
# pipeline = make_pipeline(
#     ce.OneHotEncoder(use_cat_names=True), 
#     StandardScaler(), 
#     LogisticRegression(solver='lbfgs')
# )

# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_val)
# accuracy_score(y_val, y_pred)

X_train is:        population  construction_year water_quality
55475         250               1992          soft
17576           0                  0          soft
21427         700               1999          soft
24813           0                  0          soft
13553           0                  0          soft
53494           0                  0          soft
693             0                  0          soft
53315           0                  0          soft
22086           1               2006          soft
54758           0                  0         milky
15481         208               1974          soft
5527            0                  0          soft
47627           0                  0          soft
11701          30               2006          soft
2148            1               1999          soft
49021           0                  0          soft
21353         350               1969          soft
21028           0                  0         salty
23215          60  

(       population  construction_year water_quality
 55475         250               1992          soft
 17576           0                  0          soft
 21427         700               1999          soft
 24813           0                  0          soft
 13553           0                  0          soft
 53494           0                  0          soft
 693             0                  0          soft
 53315           0                  0          soft
 22086           1               2006          soft
 54758           0                  0         milky
 15481         208               1974          soft
 5527            0                  0          soft
 47627           0                  0          soft
 11701          30               2006          soft
 2148            1               1999          soft
 49021           0                  0          soft
 21353         350               1969          soft
 21028           0                  0         salty
 23215      

In [10]:
# examine log_reg and its methods

In [11]:
# the pseudo-random baseline

print('id', ',', 'status_group')
pumps = X['id']
for pump in pumps[:10]:
    if random.choice(['not_functional', 'functional']) == 'functional':
        print(pump, ',', 'functional')

id , status_group


KeyError: 'id'

In [ ]:
# the simple LogReg baseline for Kaggle csv
# predicted_non_abandoned_pump_count = 0
max_rows = 0

print('id', ',', 'status_group')
pumps = X_test['id']
while max_rows < 14359:
    for pump, pred in zip(pumps, y_pred):
        if pred != 'unknown' and 'abandoned' not in pred and pred != 'salty':
            print(pump, ',', 'functional')
            # predicted_non_abandoned_pump_count += 1
            max_rows += 1

# print('predicted number of unabandoned pumps is:', predicted_non_abandoned_pump_count)
print('number of rows generated is:', max_rows)


In [ ]:
# for further modeling

# faulty_pumps = predict(df1.id, kernel='sigmoid', degree=1, coef0=log_reg.coef_)
# faulty_pumps

In [ ]:
# write LogReg baseline data to csv on local machine, in preparation to push to GitHub then Kaggle
prediction_csv_path = 'C:\\Users\\jhump\\Desktop\\Desktop_professional\\LSDS\\Full_Course\\kaggle-prediction.csv'

with open(prediction_csv_path, 'w') as infile:
    max_rows = 0
    infile.write('id' + ', ' + 'status_group' + '\n')
    pumps = X_test['id']
    while max_rows < 14359:
        for pump, pred in zip(pumps, y_pred):
            if 'abandoned' not in pred and 'unknown' not in pred:
                infile.write(str(pump) + ', ' + 'functional' + '\n')
                max_rows += 1
    infile.close()


In [ ]:
# attempt to understand tanzania-data-kaggle.csv vis a vis Kaggle submission req's
# command line syntax CREDIT to Ryan Herr/LSDS
tanzania_data_kaggle = 'C:\\Users\\jhump\\Desktop\\Desktop_professional\\LSDS\\Full_Course\\tanzania-data-kaggle.csv'


In [ ]:
# 2019-02-06 Based on feedback from Ryan Herr/LSDS, will refactor y above

In [ ]:
# foxtrot: analyze and interpret model predictions